In [ ]:
%%capture
!pip install -r ../requirements.txt

In [1]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Add it to sys.path if not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [3]:
from notion_client import Client
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from twilio.rest import Client as TwilioClient
from typing import TypedDict
import os
from tools.notion_fetch_agent_tools import fetch_latest_notion_journaling_entry
from typing import TypedDict
from langgraph.graph import END, StateGraph

In [43]:
# TEMPLATE ENV VARIABLES
GRAPH_ENTRY_QUERY = "Send me bible scripture that will help me with how i want to grow from my latest journaling in Notion."

In [4]:
llm = ChatOpenAI(model="gpt-4o")

In [5]:
class State(TypedDict):
    query: str
    notion_data_action: str # TEMPLATE VAR
    notion_next_node_instructions: str # TEMPLATE VAR
    notion_journal_growth_summary: str # TEMPLATE VAR
    message: str # TEMPLATE VAR

## Agent 1 - Notion

In each agent (agents/), we can have the this code as attributes/methods

In [6]:
from langchain.agents import initialize_agent

In [7]:
tools = [fetch_latest_notion_journaling_entry]

In [8]:
notion_prompt_template = PromptTemplate(
    input_variables=["notion_data_action"],
    template="You are an assistant responsible for simply making a call to the notion client API, and then doing the following instructions on the returned data: {notion_data_action}"
)

In [9]:
notion_agent = initialize_agent(
    tools=tools,
    llm=llm,
    handle_parsing_errors=True
)

/var/folders/1p/k8wcs51j0bndgtngv7p3gbsr0000gn/T/ipykernel_79557/1288154412.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  notion_agent = initialize_agent(


In [10]:
result = notion_agent.invoke(notion_prompt_template.format(notion_data_action="From this journaling, extract out the ways the user desires to grow."))

In [11]:
result

{'input': 'You are an assistant responsible for simply making a call to the notion client API, and then doing the following instructions on the returned data: From this journaling, extract out the ways the user desires to grow.',
 'output': "The user's desires to grow include: \n\n1. Spiritual Growth and Community Building: The user wants to develop their spiritual life and be used by God, seeking community among mature, godly individuals, and considering leading a group outside of a traditional church setting.\n\n2. Leadership and Influence: The user aims to harness their leadership abilities to inspire and support others, starting small and potentially expanding into a large, influential community.\n\n3. Personal Development: They wish to understand their unique purpose, rooted in intrinsic motivation, while fostering a vision and passion for serving others.\n\n4. Perseverance and Resilience: The user acknowledges growth requires enduring challenges and developing character.\n\n5. Lo

In [12]:
def notion_node(state: State) -> State:
    notion_data_action = state.get("notion_data_action")
    
    prompt = notion_prompt_template.format(notion_data_action=notion_data_action)
    result = notion_agent.invoke(prompt)
    return {"notion_journal_growth_summary": result.get('output')}

## Agent 2 - Scripture (generation)

In [13]:
generation_agent_instructions = """
You are a generalist assistant reponsible for performing a simple query. The 
instructions are:

{instructions}
"""

In [14]:
generation_agent_prompt_template = PromptTemplate(
    # TEMPLATE VAR (output of notion node or other things)
    input_variables=["instructions"],
    # TEMPLATE VAR
    template=generation_agent_instructions
)

In [15]:
generation_agent_llm_chain = LLMChain(llm=llm, prompt=generation_agent_prompt_template)

/var/folders/1p/k8wcs51j0bndgtngv7p3gbsr0000gn/T/ipykernel_79557/2322471595.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  generation_agent_llm_chain = LLMChain(llm=llm, prompt=generation_agent_prompt_template)


In [16]:
# Testing
x = "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: {notion_journal_growth_summary}"
x = generation_agent_prompt_template.format(instructions=x).format(notion_journal_growth_summary="The user desires to grow spiritually, as a leader, by embracing intrinsic motivation, strengthening relationships, maintaining joy and gratitude, and through personal accountability and a growth mindset.")
generation_agent_llm_chain.run(x)

/var/folders/1p/k8wcs51j0bndgtngv7p3gbsr0000gn/T/ipykernel_79557/3269216885.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  generation_agent_llm_chain.run(x)


'A scripture that encompasses themes of spiritual growth, leadership, intrinsic motivation, strong relationships, joy, gratitude, personal accountability, and a growth mindset is Philippians 4:13: \n\n"I can do all things through Christ who strengthens me." \n\nThis verse encourages reliance on spiritual strength, fostering leadership and motivation, and supports perseverance and personal growth.'

In [17]:
def generation_node(state: State) -> State:
    # Output from previous node
    # TEMPLATE VAR
    notion_journal_growth_summary = state.get("notion_journal_growth_summary")
    # TEMPLATE VAR
    notion_next_node_instructions = state.get("notion_next_node_instructions") + notion_journal_growth_summary
    
    prompt = generation_agent_prompt_template.format(instructions=notion_next_node_instructions)
    output = generation_agent_llm_chain.run(prompt)

    return {"message": output}

## Agent 3 - Twilio

In [18]:
from langchain.tools import tool

TO_NUM = "xyz" # TEMPLATE VAR
FROM_NUM = "xyz" # ENV VAR for the job (set on Twilio console)

@tool
def text_user(msg: str):
    """Tool to text the user via Twilio."""
    # account_sid = os.getenv("TWILIO_ACCOUNT_SID") # TODO: get these
    # auth_token = os.getenv("TWILIO_AUTH_TOKEN")
    # from_number = os.getenv("TWILIO_FROM_NUMBER")
    # client = Client(account_sid, auth_token)
    # sms = client.messages.create(
    #     body=msg,
    #     from_=from_num,
    #     to=to_num
    # )
    #return sms.sid
    print(f"Sent {msg}")

In [19]:
twilio_prompt_template = PromptTemplate(
    # TEMPLATE VAR (output of notion node or other things)
    input_variables=["message"],
    # TEMPLATE VAR
    template="Your sole purpose is to send this text message to the user: \"{message}\""
)

In [20]:
tools = [text_user]
twilio_agent = initialize_agent(
    tools=tools,
    llm=llm,
    handle_parsing_errors=True
)

In [21]:
def twilio_node(state: State) -> State:
    output = twilio_agent.invoke(twilio_prompt_template.format(message=state.get("message")))
    return {}

## Graph

In [22]:
# Initialize the graph with the state schema
graph = StateGraph(State)

# Add nodes
# TEMPLATE VAR - We can loop through the nodes in the actual app
graph.add_node("notion_node", notion_node)
graph.add_node("generation_node", generation_node)
graph.add_node("twilio_node", twilio_node)

# Set the entry point
# TEMPLATE VAR
graph.set_entry_point("notion_node")

# Define sequential edges
# TEMPLATE VAR -Also loop through this
graph.add_edge("notion_node", "generation_node")

graph.add_edge("generation_node", END)
graph.add_edge("generation_node", "twilio_node")
graph.add_edge("twilio_node", END)

compiled_graph = graph.compile()

In [24]:
# This initial state is going to contain the injected variables from the orchestrator
initial_state = {
    "query": "asdas",
    "notion_data_action": "From this journaling, extract out the primary way the user desires to grow in his/her life.",
    "notion_next_node_instructions": "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: "
}
try:
    result = compiled_graph.invoke(initial_state)
    print("Final state:", result)
except Exception as e:
    print(f"Graph-level error: {e}")

Sent A Bible verse that aligns with the user's desire for personal growth in understanding their purpose, enhancing their faith, and exercising servant leadership is: Philippians 2:3-4 (NIV): 'Do nothing out of selfish ambition or vain conceit. Rather, in humility value others above yourselves, not looking to your own interests but each of you to the interests of the others.' This verse encourages humility, selflessness, and a focus on serving others, which aligns well with the goals of spiritual grounding and servant leadership.
Final state: {'query': 'asdas', 'notion_data_action': 'From this journaling, extract out the primary way the user desires to grow in his/her life.', 'notion_next_node_instructions': "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: ", 'notion_journal_growth_summary': 'The user desires to grow by understanding his purpose, enhancing his faith, and exercising servant leadershi